# Modelo de Identificação de Fraudes

- Informações da base:
    - 'Time' (tempo): contém os segundos decorridos entre cada transação e a primeira transação no conjunto de dados. 
    - 'Amount' (valor): é o valor da transação 
    - 'Class' (classe): é a variável de resposta e assume valor 1 em caso de fraude e 0 caso contrário.
    - "Infelizmente, devido a questões de confidencialidade, não podemos fornecer os recursos originais e mais informações básicas sobre os dados. Características V1, V2, … V28 são os principais componentes obtidos com PCA"

In [1]:
# Configuração do ambiente

# Garante que o notebook está na raiz do projeto
%cd .. 

# Verifica o diretório atual (Linux/Mac)
# !pwd  

# Verifica o diretório atual (Windows)
!cd 

C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude
C:\Users\flavi\Documents\GitHub\Projeto_4_Modelo_de_Identificacao_de_Fraude


In [2]:
# Importações necessárias 

import sys
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import datetime as dt
import xgboost as xgb

from scipy.stats import kstest, normaltest, anderson

from sklearn.preprocessing import PowerTransformer, MinMaxScaler

from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree, metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC

from sklearn.feature_selection import SelectKBest, f_classif

from sklearn.model_selection import train_test_split, GridSearchCV
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids, NearMiss 
from imblearn.over_sampling import RandomOverSampler, SMOTE, ADASYN
from imblearn.combine import SMOTEENN

from sklearn.metrics import (
    confusion_matrix, 
    accuracy_score, 
    precision_score, 
    recall_score,
    precision_recall_curve,
)

import warnings
from src.config.paths import DADOS_CREDICARD, DADOS_CREDICARD_TRATADO
from src.config.auxiliares_ml import downcast_dataframe, testar_modelos_com_undersampling

warnings.filterwarnings('ignore')
sns.set_theme(palette="bright")

In [3]:
# Importando a base

transacoes = pd.read_parquet(DADOS_CREDICARD_TRATADO)

In [4]:
# Visualizando estatísticas descritivas da base 

transacoes.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,...,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.867188,1.339397e-08,0.000000,-4.800400e-08,6.643411e-09,2.657364e-08,-1.500125e-09,-1.071518e-09,-1.071518e-10,-3.214554e-10,...,-1.928732e-09,3.643161e-09,3.214554e-10,1.259033e-09,2.143036e-09,1.034684e-09,1.138488e-10,-1.272427e-10,88.349619,0.001727
std,47487.195312,1.958611e+00,1.651183,1.516203e+00,1.415832e+00,1.380180e+00,1.332214e+00,1.237020e+00,1.194202e+00,1.098575e+00,...,7.344701e-01,7.256873e-01,6.244100e-01,6.056016e-01,5.212620e-01,4.822155e-01,4.035722e-01,3.300574e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-72.715729,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,...,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-0.598550,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,...,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,0.065486,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,...,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097605e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,0.803724,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273458e-01,5.971390e-01,...,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,22.057730,9.382559e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,...,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519588e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


### Pré-processamento: 

- Utilizaremos o **PowerTransformer** devido ao grande número de outliers, **xceto em 'Time'**. <br>
- Em **'Time'** utilizaremos o **MinMaxScaler**. <br>
- Vamos utilizar o **RandomUnderSampler** para balancear nossa base.

In [5]:
# Criando os escaladores

power_transformer = PowerTransformer(method='yeo-johnson')  # Funciona para valores positivos e negativos
minmax_scaler = MinMaxScaler()

# Aplicando a normalização nas colunas específicas

transacoes['Amount'] = power_transformer.fit_transform(transacoes[['Amount']])
transacoes['Time'] = minmax_scaler.fit_transform(transacoes[['Time']])


transacoes.head(2)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,1.119013,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,-1.128464,0


In [6]:
# Separando X e y

X = transacoes.drop(columns='Class')
y = transacoes['Class']

### Fazendo Cross-validation com StratifiedKFold e utilizando o RandomUnderSampler

In [7]:
# Configuração do StratifiedKFold

cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=0)

# Criando o RandomUnderSampler

rus = RandomUnderSampler(random_state=0)

# Lista para armazenar os conjuntos de treino e teste de cada dobra

folds_data = []

for train_idx, test_idx in cv.split(X, y):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Aplicando o balanceamento com RandomUnderSampler nos dados de treino
    
    X_train_bal, y_train_bal = rus.fit_resample(X_train, y_train)

    # Armazenando os dados da dobra
    
    folds_data.append((X_train_bal, y_train_bal, X_test, y_test))

print("Cross-validation com balanceamento concluído. Agora podemos testar os modelos.")

Cross-validation com balanceamento concluído. Agora podemos testar os modelos.


### XGBoost 

In [8]:
# Criando o modelo XGBoost

modelo_xgb = xgb.XGBClassifier(
    max_depth=9,
    n_estimators=200,
    learning_rate=0.3,
    random_state=0,
    scale_pos_weight=(y.value_counts()[0] / y.value_counts()[1])  # Lida com desbalanceamento
)

# Loop pelas dobras do cross-validation

for i, (X_train_bal, y_train_bal, X_test, y_test) in enumerate(folds_data):
    modelo_xgb.fit(X_train_bal, y_train_bal)  # Treina no conjunto balanceado
    y_pred_xgb = modelo_xgb.predict(X_test)  # Testa no conjunto original
    
    # Calculando métricas
    
    acc = accuracy_score(y_test, y_pred_xgb)
    prec = precision_score(y_test, y_pred_xgb, zero_division=0)  # zero_division evita erros se não houver positivos
    rec = recall_score(y_test, y_pred_xgb)

    # Exibindo os resultados
    
    print(f"Dobra {i+1}:")
    print(f"  Acurácia:  {acc:.4f}")
    print(f"  Precisão:  {prec:.4f}")
    print(f"  Recall:    {rec:.4f}\n")

Dobra 1:
  Acurácia:  0.9108
  Precisão:  0.0178
  Recall:    0.9329

Dobra 2:
  Acurácia:  0.9028
  Precisão:  0.0164
  Recall:    0.9390

Dobra 3:
  Acurácia:  0.9064
  Precisão:  0.0171
  Recall:    0.9390



### Random Forest

In [9]:
# Utilizando o Random Forest

clf_RF = RandomForestClassifier(
    max_depth=7,  
    random_state=0,
    criterion='gini',
    n_estimators=100,  
    class_weight='balanced_subsample' 
).fit(X_train, y_train)

# Loop pelas dobras do cross-validation

for i, (X_train_bal, y_train_bal, X_test, y_test) in enumerate(folds_data):
    clf_RF.fit(X_train_bal, y_train_bal)  # Treina no conjunto balanceado
    y_pred_RF = clf_RF.predict(X_test)  # Testa no conjunto original
    
    # Calculando métricas
    
    acc = accuracy_score(y_test, y_pred_RF)
    prec = precision_score(y_test, y_pred_RF, zero_division=0)  # zero_division evita erros se não houver positivos
    rec = recall_score(y_test, y_pred_RF)

    # Exibindo os resultados
    
    print(f"Dobra {i+1}:")
    print(f"  Acurácia:  {acc:.4f}")
    print(f"  Precisão:  {prec:.4f}")
    print(f"  Recall:    {rec:.4f}\n")

Dobra 1:
  Acurácia:  0.9828
  Precisão:  0.0821
  Recall:    0.8780

Dobra 2:
  Acurácia:  0.9857
  Precisão:  0.0995
  Recall:    0.9024

Dobra 3:
  Acurácia:  0.9667
  Precisão:  0.0449
  Recall:    0.9024



### OBSERVAÇÃO:

**Utilizamos apenas os dois algoritmos que se saíram melhor até aqui:**.
- XGBoost
- Random Forest

### NOTA:

Observamos que sempre que executamos o balanceamento da base, os resultados não são satisfatórios.